# Extract the quadrat from all GLNI images in the dataset

This is a preprocessing step for generating predictions on GLNI 2019 data.

### To successfully extract a quadrat the algorithm assumes:

- All four corners of the quadrat are contained in the image, i.e.~the side lengths or corners are not 
cropped from the scene.

- The quadrat side lengths are not occluded, e.g., by diver equipment, mesh bags, or vegetation.

- The image is sufficiently clear, i.e.~not turbulent or cloudy from disrupted sediment.

- The camera angle is within $65-90^{\circ}$ with respect to the top surface of the quadrat.
Note that this is separate from quadrat rotation in the camera plane, which can be arbitrary.

- The camera is not too far from the quadrat such that the side lengths are less than 400 pixels 
for $1080 \times 1440$ resolution, 500 pixels for $1080 \times 1920$ HD resolution, or 400 for portrait mode in HD res.

The algorithm still works reasonably well in some cases even when the assumptions are violated, e.g., input 7 with the mesh bag covering one of the corners, as missing corner coordinates can sometimes be inferred if enough complementary lines are detected. Conversely, even when the assumptions are satisfied, a best effort is made to extract the *interior* of the quadrat, but this occaisionally won't be possible due to missing or misleading lines and part of the quadrat may be included in the output.

Prior to running this notebook you must set:

1. The `DRAW` variable annotates intermediate results on the image and is nice for visualizing results. It 
should be set to False for saving the final output.

2. The default values for all other parameters can be left as they are.

When `DRAW=True`, you will see annotations in the __Post-Processing__ pane.
- Large green circles are corner centroids found by K-means
- Medium blue circles are all line intersection points after the final stage of processing
- Small white circles are proposed crop locations (the quadrat interior corner points)
- The X lines corresponding to 'Using X of Y lines' after rejecting outliers and merging similar lines are shown in blue

On completion you should have 26 cropped images in the folder `My Drive/Data/Quadrat_Extraction_from_Stills`

Dependencies:
- `opencv.__version__==4.2.0`
- `skimage.__version__==0.16.2` 

In [ ]:
DRAW = False

In [ ]:
import os
import os.path as osp
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'
    SAVE_PATH = osp.join(DATA_PATH, 'Quadrat_Extraction_from_Stills')
    
    # cd into current directory so local imports work
    %cd '/content/drive/My Drive/cciw-zebra-mussel/quadrat-extraction/'
else:
    DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')
    SAVE_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data/Videos_and_stills/TestingSet/GLNI_Quadrats/v4')
    
print('Reading data from', DATA_PATH)    
print('Saving cropped images to', SAVE_PATH)    

In [ ]:
import cv2
import numpy as np

import time
from glob import glob

# local import
from utils import crop_still_image_no_rotate, compute_pairwise_distances

import matplotlib.pyplot as plt

In [ ]:
# Search for all still files in the datset
stills = glob(os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/GLNI/*/*/*/*/*/*.jpg'))
print('Found %d still images' % len(stills))

In [ ]:
# indices of data in x and y position respectively
X, Y = 0, 1

In [ ]:
scale_percent = 25

In [ ]:
#i = 0
#bgr = cv2.imread(stills[i])
#plt.imshow(bgr)

In [ ]:
"""x_trim and y_trim are used to remove black padding
for 1080x1440 video which triggers spurious edges"""

"""@param canny_thresh initial hysteresis values for Canny edge 
detector, input to HoughLines"""

"""@param threshold Accumulator threshold, return 
lines with more than threshold of votes. (intersection points)"""
threshold = 125

"""@param minLineLength Minimum line length. 
Line segments shorter than that are rejected. (pixels)"""

"""@param maxLineGap Maximum allowed gap between points 
on the same line to link them. (pixels)"""

In [ ]:
mode = 0

for i in range(len(stills)):

    out_file = stills[i].split('/')[-1].split('.')[0] + '_crop.jpg'
    if osp.exists(osp.join(SAVE_PATH, out_file)):
        continue

    # load the input image
    bgr = cv2.imread(stills[i])

    # case 1 of 4: small resolution, portrait mode
    if bgr.shape[1] == 3788:
        print(i, ' case 1 of 4: small resolution, portrait mode')
        x_trim, y_trim = 10, 1200
        #x_trim, y_trim = 200, 1100
        canny_thresh1, canny_thresh2 = 10, 50
        mLL, mLG = 400, 200    
        mode = 1

    # case 2 of 4: small resolution, landscape mode
    elif bgr.shape[1] == 6738:
        print(i, ' case 2 of 4: small resolution, landscape mode')
        x_trim, y_trim = 1000, 1
        #x_trim, y_trim = 1100, 200
        canny_thresh1, canny_thresh2 = 10, 50
        mLL, mLG = 400, 200
        mode = 2

    # case 3 of 4: large resolution, portrait mode
    elif bgr.shape[1] == 4924:
        print(i, ' case 3 of 4: large resolution, portrait mode')
        x_trim, y_trim = 100, 1100
        canny_thresh1, canny_thresh2 = 10, 50
        mLL, mLG = 450, 225
        mode = 3

    # case 4 of 4: large resolution, landscape mode
    elif bgr.shape[1] == 7378:
        x_trim, y_trim = 1100, 100
        print(i, ' case 4 of 4: large resolution, landscape mode')
        canny_thresh1, canny_thresh2 = 10, 50
        mLL, mLG = 450, 225    
        mode = 4

    #bgr = bgr[y_trim:-y_trim, x_trim:-x_trim, :]    
    width = int(bgr.shape[1] * scale_percent / 100)
    height = int(bgr.shape[0] * scale_percent / 100)    
    bgr = cv2.resize(bgr, (width, height)) # resize image

    '''
    plt.close('all')
    plt.figure()
    plt.imshow(bgr)
    plt.title('Input %d' % i, fontsize=24)
    plt.show()
    '''

    start_time = time.time()
    bgr, edges, crop = crop_still_image_no_rotate(
        bgr, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)
    print('Processing took %.2f sec' % float(time.time() - start_time))

    '''
    plt.figure()
    plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
    plt.title('Post-Processing', fontsize=24)
    plt.show()
    '''

    try:
        x_start = crop[:, X].min()
        x_end = crop[:, X].max()
        y_start = crop[:, Y].min()
        y_end = crop[:, Y].max()

        if (compute_pairwise_distances(crop)[:, 2] < mLL).sum():
            print('Corners do not form a square, try to crop based on two clusters')
            if mode == 2 or mode == 4:
                '''
                centroid_y = crop[:, Y].mean()
                d_from_y_mean = np.abs(crop[:, Y] - centroid_y)
                crop_wrt_y = crop[np.argsort(d_from_y_mean)][:2]       
                x_ref = int(crop_wrt_y[:, X].mean())
                y_start = crop_wrt_y[:, Y].min()
                y_end = crop_wrt_y[:, Y].max()
                '''
                centroid, crop, cluster_centers = centroid_and_crop_pts_k2means(crop)

                # call this x_ref because we don't know if it's left or right
                x_ref = int(crop[:, X].mean())
                y_start = crop[:, Y].min()
                y_end = crop[:, Y].max()
                delta = y_end - y_start

                if (x_ref + delta) > bgr.shape[1]:
                    x_start = x_ref - delta
                    x_end = x_ref
                else:
                    x_end = x_ref + delta
                    x_start = x_ref
            else:
                '''
                centroid_x = crop[:, X].mean()
                d_from_x_mean = np.abs(crop[:, X] - centroid_x)
                crop_wrt_x = crop[np.argsort(d_from_x_mean)][:2]       
                y_ref = int(crop_wrt_x[:, Y].mean())
                x_start = crop_wrt_x[:, X].min()
                x_end = crop_wrt_x[:, X].max()
                '''            
                centroid, crop, cluster_centers = centroid_and_crop_pts_k2means(crop)
                y_ref = int(crop[:, Y].mean())
                x_start = crop[:, X].min()
                x_end = crop[:, X].max()

                # call this y_ref because we don't know if it's top or bottom
                delta = x_end - x_start

                if (y_ref + delta) > bgr.shape[0]:
                    y_start = y_ref - delta
                    y_end = y_ref
                else:
                    y_end = y_ref + delta
                    y_start = y_ref

        if (y_end - y_start) > mLL and (x_end - x_start) > mLL:
            try:
                '''
                plt.figure()
                plt.imshow(bgr[y_start:y_end, x_start:x_end, :])
                '''
                cv2.imwrite(osp.join(SAVE_PATH, out_file), bgr[y_start:y_end, x_start:x_end, :])
                print(i, 'SUCCESS!')
            except:
                print('Cannot write ', out_file)
        else:
            print('Corners too close to crop')
    except:
        print('Cannot crop: insufficient number of corner points found')

In [ ]:
stills[:10]

In [ ]:
plt.imshow(bgr[y_start:y_end, x_start:x_end, :])

In [ ]:
crop

In [ ]:
x_end

In [ ]:
centroid, crop, cluster_centers = centroid_and_crop_pts_k2means(crop)

In [ ]:
centroid

In [ ]:
crop[:, X].mean()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def centroid_and_crop_pts_k2means(corners):
    """Use K-means clustering to optimally find the 
    image centroid and four crop points given 
    at least four corner points"""
    
    kmeans = KMeans(n_clusters=2, random_state=0).fit(corners)
    
    centroid = kmeans.cluster_centers_.mean(axis=0)
    
    crop = []
    for i in range(2):
        cluster = corners[kmeans.labels_ == i]
        intra_cluster_d2centroid = np.linalg.norm(cluster - centroid, axis=1)
        indices = np.argsort(intra_cluster_d2centroid)
        crop.append(cluster[indices][0].astype('int'))
    crop = np.asarray(crop)
    return centroid, crop, kmeans.cluster_centers_

In [ ]:
centroid_y = crop[:, Y].mean()
d_from_y_mean = np.abs(crop[:4, Y] - centroid_y)
d_from_y_mean

In [ ]:
d_from_y_mean

In [ ]:
crop_wrt_y = crop[np.argsort(d_from_y_mean)][:2]
crop_wrt_y

In [ ]:
d_from_y_mean

In [ ]:
y_end

In [ ]:
x_ref + delta

In [ ]:
#bgr.shape[1]

In [ ]:
x_ref

In [ ]:
delta

In [ ]:
x_start

In [ ]:
x_end

In [ ]:
y_end

In [ ]:
y_start

In [ ]:
y_ref

In [ ]:
x_start

In [ ]:
x_end

In [ ]:
d_from_x_mean

In [ ]:
d_from_y_mean = np.abs(crop[:, Y] - centroid_y)
d_from_y_mean

In [ ]:
crop_wrt_y = crop[np.argsort(d_from_y_mean)][:2]

In [ ]:
#crop_wrt_y
x_start = int(crop_wrt_y[:, X].mean())

In [ ]:
y_start = crop_wrt_y[:, Y].min()
y_end = crop_wrt_y[:, Y].max()
x_end = x_start + y_end - y_start
x_end

In [ ]:
compute_pairwise_distances(crop)

In [ ]:
plt.close('all')
plt.figure(figsize=(5, 20))
plt.imshow(bgr)
plt.title('Input %d' % (i + 1), fontsize=24)
plt.show()

# End demo

In [ ]:
# to plot all still images
for i in range(len(stills)):
    bgr = cv2.imread(stills[i])
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    plt.close('all')
    plt.figure(figsize=(10, 8))
    plt.imshow(bgr[::2, ::2, :])
    plt.title(i + 1)
    plt.show()

In [ ]:
'''
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig, ax1 = plt.subplots(1, 1, figsize=(8, 5))
#bgr = cv2.imread(stills[i])
#rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
ax1.imshow(img)
#ax1.imshow(edges)
#ax1.set_title('Canny edges', fontsize=16)
#ax2.imshow(bgr)
#ax2.set_title('Still image with corner points', fontsize=16)
ax1.axis('off')
#ax2.axis('off')
plt.tight_layout()
plt.show()
#fig.savefig('quatract-extraction-still-%d-step-4.pdf' % i)
'''